# Advanced classification models

This example shows how to use more advanced classifiers instead of the linear classifier that is used by default.

In [51]:
import requests
from io import BytesIO
import pprint
import numpy as np
import scipy.io
from sklearn.preprocessing import OneHotEncoder

from reservoir_computing.modules import RC_model
from reservoir_computing.utils import compute_test_scores

np.random.seed(0) # Fix the seed for reproducibility

Let's start by defining the configuration for the Reservoir, the dimensionality reduction module, and the type of Multivariate Time Series (MTS) representation.

In [52]:
config = {}

# Hyperarameters of the reservoir
config['n_internal_units'] = 450        # size of the reservoir
config['spectral_radius'] = 0.59        # largest eigenvalue of the reservoir
config['leak'] = 0.6                    # amount of leakage in the reservoir state update (None or 1.0 --> no leakage)
config['connectivity'] = 0.25           # percentage of nonzero connections in the reservoir
config['input_scaling'] = 0.1           # scaling of the input weights
config['noise_level'] = 0.01            # noise in the reservoir state update
config['n_drop'] = 5                    # transient states to be dropped
config['bidir'] = True                  # if True, use bidirectional reservoir
config['circ'] = False                  # use reservoir with circle topology

# Dimensionality reduction hyperparameters
config['dimred_method'] = 'tenpca'      # options: {None (no dimensionality reduction), 'pca', 'tenpca'}
config['n_dim'] = 75                    # number of resulting dimensions after the dimensionality reduction procedure

# Type of MTS representation
config['mts_rep'] = 'reservoir'         # MTS representation:  {'last', 'mean', 'output', 'reservoir'}
config['w_ridge_embedding'] = 10.0      # regularization parameter of the ridge regression

We will start using a simple linear classifier as the readout. In particular, we will use the [RidgeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html) from sklearn. The classifier requires to define a regularization parameter that we call `w_ridge` (but in sklearn is called `alpha`).

In [53]:
# Type of readout
config['readout_type'] = 'lin'          # readout used for classification
config['w_ridge'] = 1.0                 # regularization of the ridge regression readout

Next, we loead and pre-process a dataset of MTS representing the stroke for drawing different Japanese vowels. Note that we need to transform the labels to one-hot encoded vectors.

In [54]:
data_url = 'https://raw.githubusercontent.com/FilippoMB/Time-series-classification-and-clustering-with-Reservoir-Computing/master/dataset/JpVow.mat'
response = requests.get(data_url)
response.raise_for_status()
data = scipy.io.loadmat(BytesIO(response.content))

Xtr = data['X']  # shape is [N,T,V]
if len(Xtr.shape) < 3:
    Xtr = np.atleast_3d(Xtr)
Ytr = data['Y']  # shape is [N,1]
Xte = data['Xte']
if len(Xte.shape) < 3:
    Xte = np.atleast_3d(Xte)
Yte = data['Yte']

print(f"Loaded data from {data_url}\nData shapes:\n Tr: {Xtr.shape}\n Te: {Xte.shape}")

# One-hot encoding for labels
onehot_encoder = OneHotEncoder(sparse_output=False)
Ytr = onehot_encoder.fit_transform(Ytr)
Yte = onehot_encoder.transform(Yte)

Loaded data from https://raw.githubusercontent.com/FilippoMB/Time-series-classification-and-clustering-with-Reservoir-Computing/master/dataset/JpVow.mat
Data shapes:
 Tr: (270, 29, 12)
 Te: (370, 29, 12)


At this point we initialize the RC classifier by passing the configuration we specified before and then we fit it on the training data.

In [55]:
classifier =  RC_model(reservoir=None,     
                       n_internal_units=config['n_internal_units'],
                       spectral_radius=config['spectral_radius'],
                       leak=config['leak'],
                       connectivity=config['connectivity'],
                       input_scaling=config['input_scaling'],
                       noise_level=config['noise_level'],
                       circle=config['circ'],
                       n_drop=config['n_drop'],
                       bidir=config['bidir'],
                       dimred_method=config['dimred_method'], 
                       n_dim=config['n_dim'],
                       mts_rep=config['mts_rep'],
                       w_ridge_embedding=config['w_ridge_embedding'],
                       readout_type=config['readout_type'],            
                       w_ridge=config['w_ridge'])

# Train the model
tr_time = classifier.fit(Xtr, Ytr) 

Training completed in 0.01 min


At this point, we can predict the labels of the test set and see how much they resemble the real ones by computing the classification accuracy and the F1 score.

In [56]:
# Compute predictions on test data
pred_class = classifier.predict(Xte) 
accuracy, f1 = compute_test_scores(pred_class, Yte)
print(f"Accuracy = {accuracy:.3f}, F1 = {f1:.3f}")

Accuracy = 0.973, F1 = 0.973


That is a pretty high accuracy. Even a simple model such as the RidgeClassifier can classify almost perfectly the test data thanks to the powerful representational power of the representation provided by the RC model.

Next, we will try more classifiers more powerful than the RidgeClassifier. In this example, we do not expect to see extreme changes in the performance since the classification performance is already very high. However, in more complex tasks using a more powerful classifier can bring substantial benefits.

We will start with [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) the Support Vector Machine Classifier of sklearn.

The first thing is to define the hyperparameters of the new classifier and pass them to the RC model.

In [57]:
# Type of readout
config['readout_type'] = 'svm'          # readout used for classification
config['svm_gamma'] = 5e-3              # bandwith of the RBF kernel
config['svm_C'] = 10.0                  # regularization for SVM hyperplane

Next, we re-create the RC model, we train, and then we test it.

In [58]:
classifier =  RC_model(reservoir=None,     
                       n_internal_units=config['n_internal_units'],
                       spectral_radius=config['spectral_radius'],
                       leak=config['leak'],
                       connectivity=config['connectivity'],
                       input_scaling=config['input_scaling'],
                       noise_level=config['noise_level'],
                       circle=config['circ'],
                       n_drop=config['n_drop'],
                       bidir=config['bidir'],
                       dimred_method=config['dimred_method'], 
                       n_dim=config['n_dim'],
                       mts_rep=config['mts_rep'],
                       w_ridge_embedding=config['w_ridge_embedding'],
                       readout_type=config['readout_type'],            
                       svm_gamma=config['svm_gamma'],
                       svm_C=config['svm_C'])

# Train the model
tr_time = classifier.fit(Xtr, Ytr) 

# Compute predictions on test data
pred_class = classifier.predict(Xte) 
accuracy, f1 = compute_test_scores(pred_class, Yte)
print(f"Accuracy = {accuracy:.3f}, F1 = {f1:.3f}")

Training completed in 0.01 min
Accuracy = 0.954, F1 = 0.955


As expected, the performance is still good but not much different from the one we got earlier.

Next, we can use a simple neural network as the classifier. We will use the Multilayer Perceptron ([MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)) from sklearn.

In this case, we have more hyperparameters to tune. To find the optimal ones when dealing with a real-world application you should do a proper hyperparameter search using a validation set.

In [59]:
# Type of readout
config['readout_type'] = 'mlp'          # readout used for classification
config['mlp_layout'] = (64,32)          # neurons in each MLP layer
config['num_epochs'] = 2000             # number of epochs 
config['w_l2'] = 1e-4                   # weight of the L2 regularization
config['nonlinearity'] = 'tanh'         # type of activation function {'relu', 'tanh', 'logistic', 'identity'}

As before, we create our RC classifier, we train it and test on unseen data.

In [61]:
classifier =  RC_model(reservoir=None,     
                       n_internal_units=config['n_internal_units'],
                       spectral_radius=config['spectral_radius'],
                       leak=config['leak'],
                       connectivity=config['connectivity'],
                       input_scaling=config['input_scaling'],
                       noise_level=config['noise_level'],
                       circle=config['circ'],
                       n_drop=config['n_drop'],
                       bidir=config['bidir'],
                       dimred_method=config['dimred_method'], 
                       n_dim=config['n_dim'],
                       mts_rep=config['mts_rep'],
                       w_ridge_embedding=config['w_ridge_embedding'],
                       readout_type=config['readout_type'],            
                       mlp_layout=config['mlp_layout'],
                       num_epochs=config['num_epochs'],
                       w_l2=config['w_l2'],
                       nonlinearity=config['nonlinearity'])

# Train the model
tr_time = classifier.fit(Xtr, Ytr) 

# Compute predictions on test data
pred_class = classifier.predict(Xte) 
accuracy, f1 = compute_test_scores(pred_class, Yte)
print(f"Accuracy = {accuracy:.3f}, F1 = {f1:.3f}")

Training completed in 0.11 min
Accuracy = 0.968, F1 = 0.969


Also in this case, the classifier obtains good performance but not too different from the previous cases.

More complicated models such as SVC and an MLP requires a proper tuning but, on difficult task, can achieve better performance compared to a simple linear classifier.